# 🌾 Rice Classification | Part 1: Data Exploration & Preprocessing
**Arborio vs Jasmine — Binary Classification**

**Run order:** This notebook → Part 2 → Part 3

> 📥 **Before running:** Upload `Rice_MSC_Dataset.csv` to your Colab session
> or mount Google Drive and update the file path in Cell 2.

In [ ]:
# ── Cell 1: Imports ─────────────────────────────────────────────
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

sns.set_theme(style='whitegrid', palette='Set2')
plt.rcParams['figure.figsize'] = (10, 5)
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print('✅ Libraries ready')

In [ ]:
# ── Cell 2: Load & Filter ────────────────────────────────────────
# If using Google Drive: from google.colab import drive; drive.mount('/content/drive')
# Then update path: pd.read_csv('/content/drive/MyDrive/Rice_MSC_Dataset.csv')

full_df = pd.read_csv('Rice_MSC_Dataset.csv')

# Keep only the two varieties we care about
rice_df = full_df[full_df['CLASS'].isin(['Arborio', 'Jasmine'])].copy()

# Encode labels as numbers — ML models need numeric targets
rice_df['label'] = rice_df['CLASS'].map({'Arborio': 0, 'Jasmine': 1})

feature_columns = [c for c in rice_df.columns if c not in ['CLASS', 'label']]

print(f'Dataset shape: {rice_df.shape}')
print(f'Features ({len(feature_columns)}): {feature_columns}')
print(f'\nClass counts:\n{rice_df["CLASS"].value_counts()}')

In [ ]:
# ── Cell 3: Dataset Statistics ───────────────────────────────────
print('Feature statistics:')
print(rice_df[feature_columns].describe().round(2).T)

In [ ]:
# ── Cell 4: Class Balance Check ──────────────────────────────────
counts = rice_df['CLASS'].value_counts()

fig, axes = plt.subplots(1, 2, figsize=(11, 4))
counts.plot(kind='bar', ax=axes[0], color=['#4C72B0','#DD8452'], edgecolor='black', rot=0)
axes[0].set_title('Sample Count per Class')
axes[0].set_ylabel('Count')
for bar in axes[0].patches:
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 30,
                 f'{int(bar.get_height()):,}', ha='center', fontweight='bold')

axes[1].pie(counts, labels=counts.index, autopct='%1.1f%%',
            colors=['#4C72B0','#DD8452'], startangle=90)
axes[1].set_title('Class Distribution')

plt.suptitle('Class Balance Check', fontweight='bold')
plt.tight_layout()
plt.show()

ratio = counts.max() / counts.min()
print(f'Imbalance ratio: {ratio:.2f}x — {"✅ Balanced" if ratio < 1.5 else "⚠️ Consider balancing techniques"}')

In [ ]:
# ── Cell 5: Missing Value Check & Handling ───────────────────────
missing = rice_df[feature_columns].isnull().sum()

if missing.sum() == 0:
    print('✅ No missing values found')
else:
    for col in missing[missing > 0].index:
        pct = missing[col] / len(rice_df) * 100
        if pct < 30:
            rice_df[col] = rice_df[col].fillna(rice_df[col].median())
            print(f'Filled "{col}" with median ({pct:.1f}% missing)')
        else:
            rice_df.drop(columns=[col], inplace=True)
            feature_columns.remove(col)
            print(f'Dropped "{col}" ({pct:.1f}% missing — too high to impute)')

    print('✅ Missing value handling complete')

In [ ]:
# ── Cell 6: Outlier Detection & Capping (IQR Method) ────────────
# IQR = middle 50% of the data. Values beyond 1.5×IQR from Q1/Q3 = outlier.
# We cap (clip) rather than delete to preserve sample size.

outlier_counts = {}
for col in feature_columns:
    Q1, Q3 = rice_df[col].quantile([0.25, 0.75])
    IQR    = Q3 - Q1
    lo, hi = Q1 - 1.5*IQR, Q3 + 1.5*IQR
    n_out  = ((rice_df[col] < lo) | (rice_df[col] > hi)).sum()
    outlier_counts[col] = n_out
    rice_df[col] = np.clip(rice_df[col], lo, hi)  # Clip to fence values

print('Outliers detected and capped per feature:')
for col, count in outlier_counts.items():
    print(f'  {col}: {count}')
print(f'\nData shape unchanged: {rice_df.shape}')

In [ ]:
# ── Cell 7: Outlier Box Plots ────────────────────────────────────
n_cols = 3
n_rows = (len(feature_columns) + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 3.5))
axes = axes.flatten()

for i, feat in enumerate(feature_columns):
    rice_df.boxplot(column=feat, by='CLASS', ax=axes[i],
                    medianprops=dict(color='red', linewidth=2),
                    flierprops=dict(marker='o', markerfacecolor='orange', markersize=3, alpha=0.5))
    axes[i].set_title(feat, fontsize=9, fontweight='bold')
    axes[i].set_xlabel('')

for j in range(i+1, len(axes)): axes[j].set_visible(False)
plt.suptitle('Feature Distributions by Class (after outlier capping)', fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# ── Cell 8: EDA — Visualisation 1: Overlapping Histograms ───────
arborio = rice_df[rice_df['CLASS'] == 'Arborio']
jasmine  = rice_df[rice_df['CLASS'] == 'Jasmine']

n_rows = (len(feature_columns) + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, n_rows * 3.5))
axes = axes.flatten()

for i, feat in enumerate(feature_columns):
    axes[i].hist(arborio[feat], bins=40, alpha=0.6, label='Arborio', color='#4C72B0')
    axes[i].hist(jasmine[feat],  bins=40, alpha=0.6, label='Jasmine',  color='#DD8452')
    axes[i].set_title(feat, fontsize=9, fontweight='bold')
    axes[i].legend(fontsize=8)

for j in range(i+1, len(axes)): axes[j].set_visible(False)
plt.suptitle('Viz 1: Feature Distributions — Arborio vs Jasmine\n'
             'Separated peaks = strong discriminator | Overlapping = weak', fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# ── Cell 9: EDA — Visualisation 2: Correlation Heatmap ──────────
corr_matrix = rice_df[feature_columns].corr()
mask        = np.triu(np.ones_like(corr_matrix, dtype=bool))  # Hide upper triangle (mirror)

fig, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f',
            cmap='RdYlGn', center=0, square=True, linewidths=0.5, ax=ax)
ax.set_title('Viz 2: Feature Correlation Heatmap\n'
             'Near ±1 = redundant pair | Near 0 = independent', fontweight='bold')
plt.tight_layout()
plt.show()

# Flag highly correlated pairs
print('Highly correlated pairs (|r| > 0.90):')
found = False
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > 0.90:
            print(f'  {corr_matrix.columns[i]} ↔ {corr_matrix.columns[j]}: {corr_matrix.iloc[i,j]:.3f}')
            found = True
if not found: print('  ✅ No highly redundant pairs found')

In [ ]:
# ── Cell 10: EDA — Visualisation 3: Pairplot of Top 5 Features ──
# Rank features by effect size: how far apart are the two class means?
effect_sizes = {
    feat: abs(arborio[feat].mean() - jasmine[feat].mean()) / rice_df[feat].std()
    for feat in feature_columns
}
top5 = sorted(effect_sizes, key=effect_sizes.get, reverse=True)[:5]

print('Top 5 most discriminative features:')
for r, f in enumerate(top5, 1): print(f'  {r}. {f}: {effect_sizes[f]:.3f}')

pair_grid = sns.pairplot(
    rice_df[top5 + ['CLASS']],
    hue='CLASS',
    palette={'Arborio': '#4C72B0', 'Jasmine': '#DD8452'},
    diag_kind='kde',
    plot_kws={'alpha': 0.4, 's': 15}
)
pair_grid.figure.suptitle('Viz 3: Pairplot — Top 5 Discriminative Features', y=1.02, fontweight='bold')
plt.show()

In [ ]:
# ── Cell 11: Train / Validation / Test Split ─────────────────────
# stratify=y ensures each split has the same class ratio as the full dataset

X = rice_df[feature_columns].values
y = rice_df['label'].values

X_tv, X_test,  y_tv, y_test  = train_test_split(
    X, y, test_size=0.15, random_state=RANDOM_SEED, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(
    X_tv, y_tv, test_size=(0.15/0.85), random_state=RANDOM_SEED, stratify=y_tv)

print(f'Train : {X_train.shape[0]} samples ({X_train.shape[0]/len(y)*100:.1f}%)')
print(f'Val   : {X_val.shape[0]} samples ({X_val.shape[0]/len(y)*100:.1f}%)')
print(f'Test  : {X_test.shape[0]} samples ({X_test.shape[0]/len(y)*100:.1f}%)')

In [ ]:
# ── Cell 12: Feature Scaling ─────────────────────────────────────
# Fit ONLY on training data — never let val/test data influence the scaler

scaler          = StandardScaler()
X_train_scaled  = scaler.fit_transform(X_train)
X_val_scaled    = scaler.transform(X_val)    # Same scaler, no refit
X_test_scaled   = scaler.transform(X_test)   # Same scaler, no refit

print('After scaling — training set stats (should be ~0 mean, ~1 std):')
print(f'  Mean: {X_train_scaled.mean(axis=0)[:3].round(4)}  ...')
print(f'  Std:  {X_train_scaled.std(axis=0)[:3].round(4)}   ...')
print('✅ Scaling complete')

In [ ]:
# ── Cell 13: Save Preprocessed Data for Part 2 ───────────────────
import pickle

data = {
    'X_train': X_train_scaled, 'X_val': X_val_scaled, 'X_test': X_test_scaled,
    'y_train': y_train,        'y_val': y_val,         'y_test': y_test,
    'feature_columns': feature_columns,
    'label_mapping': {'Arborio': 0, 'Jasmine': 1},
    'scaler': scaler
}

with open('part1_data.pkl', 'wb') as f:
    pickle.dump(data, f)

print('💾 Saved to part1_data.pkl')
print('   → Open Part 2 notebook next')